In [39]:
import time
import time, psutil, os
start = time.time()
m0 = psutil.Process(os.getpid()).memory_info().rss / 1024**2

In [40]:
import re
import unidecode
import polars as pl
import pandas as pd
import numpy as np
from rapidfuzz import fuzz


# Moradas de Histórico

In [41]:
import polars as pl

ficheiro = r"C:\Users\1420844\Downloads\addressGeoLoc_export_20221006.csv"

df = pl.read_csv(
    ficheiro,
    separator=",",
    quote_char='"',
    skip_rows=1,
    has_header=False,
    new_columns=["MORADA", "CP", "LOCALIDADE", "LATITUDE", "LONGITUDE"],
    decimal_comma=True
)

pl.Config.set_tbl_formatting("UTF8_FULL")  # formato mais limpo
pl.Config.set_tbl_rows(5)                 # nº de linhas a mostrar
pl.Config.set_tbl_width_chars(120) 
pl.Config.set_tbl_cols("10_000")    



polars.config.Config

In [42]:
df.head()

MORADA,CP,LOCALIDADE,LATITUDE,LONGITUDE
str,str,str,str,str
"""RUA CESARIO VERDE LOTE 3 A DAS…","""2660""","""FRIELAS""","""38.852697456""","""-9.16971532999997"""
"""RUA DO ROXICO NR50""","""3865-110""","""FERMELA""","""40.709808191""","""-8.54878568099997"""
"""AVENIDA 13 DE MAIO N 536""","""3885-227""","""CORTEGACA OVR""","""40.94109138""","""-8.61864986999996"""
"""AV REINALDO SANTOS N 24 3 DTO""","""2675-673""","""ODIVELAS""","""38.7926955990001""","""-9.18881750599996"""
"""RUA DO BARREIRO N 547""","""4405-730""","""VILA NOVA DE GAIA""","""41.113905909""","""-8.63062512299996"""


In [43]:
# garantir que CP é string
df = df.with_columns(df["CP"].cast(pl.Utf8))

# filtrar e contar
contagem = (
    df.filter(df["CP"].str.slice(0,4).is_in(["4830", "4850"]))
      .select(pl.count("MORADA"))
      .item()
)

print("Número de moradas:", contagem)

Número de moradas: 1158


In [44]:
df_filtrado = df.filter(
    df["CP"].str.slice(0, 4).is_in(["4000"])
)

df_filtrado

MORADA,CP,LOCALIDADE,LATITUDE,LONGITUDE
str,str,str,str,str
"""RUA DA FIRMEZA 482""","""4000-216""","""PORTO""","""41.1510188030001""","""-8.60600840199993"""
"""RUA DE CAMOES NO93 5OA""","""4000-144""","""PORTO""","""41.152648013""","""-8.61019293599998"""
"""RUA DE SANTA CATARINA 1463""","""4000-458""","""PORTO""","""41.159181""","""-8.6046"""
…,…,…,…,…
"""RUA DA ALEGRIA 948 1 ESQ FRT""","""4000-040""","""PORTO""","""41.160231599""","""-8.60071781199997"""
"""RUA DE CAMOES 289 3D""","""4000""","""PORTO""","""41.15445""","""-8.610141"""


In [45]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    # converter para radianos
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    # fórmula haversine
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    R = 6371000  # raio da Terra em metros
    return R * c


# Moradas Base Nacional de Endereços (BNE) 

In [46]:
ficheiro = r"C:\Users\1420844\Downloads\df_Principal_f.csv"

df_BNE = pl.read_csv(
    ficheiro,
    separator=",",
    quote_char='"',
    skip_rows=1,
    has_header=False,
    new_columns=["ART_COD",	"ART_TIPO","ART_TITULO","ART_DESIG","PORTA_NUM","CP4","CP3","CPALF","LOCALIDADE","LONG_PORTA","LAT_PORTA"],
    decimal_comma=True
)

In [47]:
df_BNE


ART_COD,ART_TIPO,ART_TITULO,ART_DESIG,PORTA_NUM,CP4,CP3,CPALF,LOCALIDADE,LONG_PORTA,LAT_PORTA
i64,str,str,str,i64,i64,i64,str,str,str,str
61213,"""Rua""",null,"""Abraos""",49,4000,12,"""PORTO""","""Porto""","""-8.59""","""41.15"""
61213,"""Rua""",null,"""Abraos""",32,4000,12,"""PORTO""","""Porto""","""-8.59""","""41.15"""
61213,"""Rua""",null,"""Abraos""",10,4000,12,"""PORTO""","""Porto""","""-8.59""","""41.15"""
…,…,…,…,…,…,…,…,…,…,…
2147140000,"""Beco""",null,"""Passos Manuel""",null,4000,7,"""PORTO""","""Porto""","""-8.6""","""41.14"""
2147140000,"""Beco""",null,"""Passos Manuel""",null,4000,7,"""PORTO""","""Porto""","""-8.6""","""41.14"""


In [70]:
# ==========================
# 1. Conversão para pandas
# ==========================
pdf1 = df_BNE.to_pandas().copy()          # contém LATITUDE / LONGITUDE
pdf2 = df_filtrado.to_pandas().copy()     # contém LAT_PORTA / LONG_PORTA

# ==========================
# 2. Normalizar nomes das colunas (remove espaços e põe maiúsculas)
# ==========================
pdf1.columns = pdf1.columns.str.strip().str.upper()
pdf2.columns = pdf2.columns.str.strip().str.upper()



In [73]:

pdf1.columns


Index(['ART_COD', 'ART_TIPO', 'ART_TITULO', 'ART_DESIG', 'PORTA_NUM', 'CP4',
       'CP3', 'CPALF', 'LOCALIDADE', 'LONG_PORTA', 'LAT_PORTA'],
      dtype='object')

In [74]:
pdf2.columns

Index(['MORADA', 'CP', 'LOCALIDADE', 'LATITUDE', 'LONGITUDE'], dtype='object')

In [76]:
# ==========================
# 1. Carregar para pandas (fonte canónica)
# ==========================
pdf_portas = df_filtrado.to_pandas().copy()   # deve ter LAT_PORTA / LONG_PORTA
pdf_bne    = df_BNE.to_pandas().copy()        # deve ter LATITUDE / LONGITUDE

# ==========================
# 2. Normalizar nomes
# ==========================
pdf_portas.columns = pdf_portas.columns.str.strip().str.upper()
pdf_bne.columns    = pdf_bne.columns.str.strip().str.upper()

# ==========================
# 3. Auto-detecção de colunas (evita KeyError se trocados)
# ==========================
has_portas_in_pdf1 = {"LAT_PORTA","LONG_PORTA"}.issubset(set(pdf_portas.columns))
has_bne_in_pdf2    = {"LATITUDE","LONGITUDE"}.issubset(set(pdf_bne.columns))

# trocar se vierem invertidos
if not has_portas_in_pdf1 and {"LAT_PORTA","LONG_PORTA"}.issubset(set(pdf_bne.columns)):
    pdf_portas, pdf_bne = pdf_bne, pdf_portas

# validações finais (falha cedo com mensagem clara)
assert {"LAT_PORTA","LONG_PORTA"}.issubset(set(pdf_portas.columns)), "Falta LAT_PORTA/LONG_PORTA no dataset de portas."
assert {"LATITUDE","LONGITUDE"}.issubset(set(pdf_bne.columns)), "Falta LATITUDE/LONGITUDE no dataset BNE."

# ==========================
# 4. Limpeza e conversão coordenadas → float
# ==========================
def clean_coord(s):
    # trata vírgulas, strings nulas e converte para float
    return (
        s.astype(str)
         .str.replace(",", ".", regex=False)
         .replace(["None","nan","NaN",""], None)
         .astype(float)
    )

pdf_portas["LAT_PORTA"]  = clean_coord(pdf_portas["LAT_PORTA"])
pdf_portas["LONG_PORTA"] = clean_coord(pdf_portas["LONG_PORTA"])
pdf_bne["LATITUDE"]      = clean_coord(pdf_bne["LATITUDE"])
pdf_bne["LONGITUDE"]     = clean_coord(pdf_bne["LONGITUDE"])

# remover linhas sem coordenadas válidas
pdf_portas = pdf_portas.dropna(subset=["LAT_PORTA","LONG_PORTA"])
pdf_bne    = pdf_bne.dropna(subset=["LATITUDE","LONGITUDE"])

# ==========================
# 5. Produto cartesiano (nota: O(N*M))
# ==========================
pdf_portas["KEY"] = 1
pdf_bne["KEY"]    = 1
pairs = pdf_portas.merge(pdf_bne, on="KEY", suffixes=("_PORTA","_BNE")).drop(columns="KEY")

# ==========================
# 6. Haversine vectorizado (metros)
# ==========================
import numpy as np

def haversine_vec(lat1, lon1, lat2, lon2):
    # inputs em graus
    R = 6371000.0  # raio da Terra em m
    p = np.pi/180.0
    dlat = (lat2-lat1)*p
    dlon = (lon2-lon1)*p
    a = np.sin(dlat/2.0)**2 + np.cos(lat1*p)*np.cos(lat2*p)*np.sin(dlon/2.0)**2
    return 2*R*np.arcsin(np.sqrt(a))

pairs["dist_m"] = haversine_vec(
    pairs["LAT_PORTA_PORTA"].to_numpy(),
    pairs["LONG_PORTA_PORTA"].to_numpy(),
    pairs["LATITUDE_BNE"].to_numpy(),
    pairs["LONGITUDE_BNE"].to_numpy(),
)

# ==========================
# 7. Filtro ≤ 500 m
# ==========================
pairs_500 = pairs.loc[pairs["dist_m"] <= 500].reset_index(drop=True)

# ==========================
# 8. Output
# ==========================
pairs_500


MemoryError: Unable to allocate 15.3 GiB for an array with shape (2052930419,) and data type int64

In [77]:
# ==========================
# 1. Conversão para pandas
# ==========================
pdf_portas = df_filtrado.to_pandas().copy()   # tem LAT_PORTA / LONG_PORTA
pdf_bne    = df_BNE.to_pandas().copy()        # tem LATITUDE / LONGITUDE

# ==========================
# 2. Normalizar nomes
# ==========================
pdf_portas.columns = pdf_portas.columns.str.strip().str.upper()
pdf_bne.columns    = pdf_bne.columns.str.strip().str.upper()

# ==========================
# 3. Limpeza e conversão coordenadas → float
# ==========================
def clean_coord(col):
    return (
        col.astype(str)
           .str.replace(",", ".", regex=False)
           .replace(["None", "nan", "NaN", ""], None)
           .astype(float)
    )

pdf_portas["LAT_PORTA"]  = clean_coord(pdf_portas["LAT_PORTA"])
pdf_portas["LONG_PORTA"] = clean_coord(pdf_portas["LONG_PORTA"])
pdf_bne["LATITUDE"]      = clean_coord(pdf_bne["LATITUDE"])
pdf_bne["LONGITUDE"]     = clean_coord(pdf_bne["LONGITUDE"])

pdf_portas = pdf_portas.dropna(subset=["LAT_PORTA","LONG_PORTA"])
pdf_bne    = pdf_bne.dropna(subset=["LATITUDE","LONGITUDE"])

# ==========================
# 4. Preparar coordenadas em radianos (obrigatório p/ BallTree haversine)
# ==========================
import numpy as np
from sklearn.neighbors import BallTree

coords_bne    = np.radians(pdf_bne[["LATITUDE","LONGITUDE"]].values)
coords_portas = np.radians(pdf_portas[["LAT_PORTA","LONG_PORTA"]].values)

# ==========================
# 5. Criar BallTree e consultar vizinhos num raio de 500 m
# ==========================
EARTH_RADIUS = 6371000  # raio da Terra em metros
tree = BallTree(coords_bne, metric="haversine")

# Raio máximo em radianos
radius = 500 / EARTH_RADIUS

# Para cada porta → devolve índices das moradas BNE dentro de 500 m
indices = tree.query_radius(coords_portas, r=radius)

# ==========================
# 6. Construir DataFrame resultado
# ==========================
matches = []
for i, idx_list in enumerate(indices):
    if len(idx_list) == 0:
        continue  # nenhuma morada próxima
    porta = pdf_portas.iloc[i]
    moradas = pdf_bne.iloc[idx_list]
    moradas = moradas.assign(
        ID_PORTA=porta.get("PORTA_NUM", np.nan),
        ART_COD=porta.get("ART_COD", np.nan),
        DIST_M=haversine(
            porta["LAT_PORTA"], porta["LONG_PORTA"],
            moradas["LATITUDE"], moradas["LONGITUDE"]
        )
    )
    matches.append(moradas)

pairs_500 = pd.concat(matches, ignore_index=True)

# ==========================
# 7. Visualização
# ==========================
pairs_500.head()


KeyError: 'LAT_PORTA'

In [ ]:
!pip install scikit-learn --upgrade


In [53]:
# ==========================
# 1. Conversão para pandas
# ==========================
pdf1 = df_BNE.to_pandas().copy()         # contém LATITUDE / LONGITUDE
pdf2 = df_filtrado.to_pandas().copy()    # contém LAT_PORTA / LONG_PORTA

# ==========================
# 2. Substituir vírgulas e converter coordenadas para float
# ==========================
pdf1["LATITUDE"] = pdf1["LATITUDE"].str.replace(",", ".").astype(float)
pdf1["LONGITUDE"] = pdf1["LONGITUDE"].str.replace(",", ".").astype(float)

pdf2["LAT_PORTA"] = pdf2["LAT_PORTA"].str.replace(",", ".").astype(float)
pdf2["LONG_PORTA"] = pdf2["LONG_PORTA"].str.replace(",", ".").astype(float)

# ==========================
# 3. Criar produto cartesiano (todas as combinações)
# ==========================
pdf1["key"] = 1
pdf2["key"] = 1
pairs = pdf1.merge(pdf2, on="key", suffixes=("_df1", "_df2")).drop("key", axis=1)

# ==========================
# 4. Calcular distância (em metros)
# ==========================
pairs["dist_m"] = haversine(
    pairs["LATITUDE_df1"], pairs["LONGITUDE_df1"],
    pairs["LAT_PORTA_df2"], pairs["LONG_PORTA_df2"]
)

# ==========================
# 5. Filtrar distâncias ≤ 500 m
# ==========================
pairs_500 = pairs[pairs["dist_m"] <= 500].reset_index(drop=True)

# ==========================
# 6. Visualização
# ==========================
pairs_500


KeyError: 'LATITUDE'

In [ ]:

# ==========================
# 1. Dicionários
# ==========================
abreviaturas = {
    "cmdt": "comandante", "cmte": "comandante",
    "dr": "doutor", "dra": "doutora",
    "sr": "senhor", "sra": "senhora", "srª": "senhora",
    "eng": "engenheiro", "enga": "engenheira", "engª": "engenheira",
    "prof": "professor", "profa": "professora", "profª": "professora",
    "arq": "arquiteto", "arqa": "arquiteta", "arqª": "arquiteta",
    "cap": "capitao", "maj": "major", "gen": "general",
    "ten": "tenente", "alm": "almirante",
    "sta": "santa", "sto": "santo", "s": "sao", "sao": "sao",
    "ns": "nossa senhora", "n s": "nossa senhora",
    "nsr": "nosso senhor", "n sr": "nosso senhor",
    "dom": "dom", "d": "dom",
    "visc": "visconde", "cond": "conde", "marq": "marques", "bar": "barao"
}

mapa_logradouros = {
    "r": "rua", "rua": "rua", "rª": "rua", "ruela": "ruela",
    "av": "avenida", "avd": "avenida", "avda": "avenida", "avenida": "avenida",
    "tv": "travessa", "trs": "travessa", "trav": "travessa", "travessa": "travessa",
    "pc": "praca", "pç": "praca", "pr": "praca", "prç": "praca", "praca": "praca",
    "lg": "largo", "largo": "largo",
    "al": "alameda", "alameda": "alameda",
    "bq": "beco", "beco": "beco",
    "esc": "escadas", "escadinha": "escadinha", "escadaria": "escadaria",
    "estr": "estrada", "estrada": "estrada",
    "cm": "caminho", "caminho": "caminho", "cam": "caminho",
    "cç": "calcada", "calcada": "calcada", "calçada": "calcada",
    "qt": "quinta", "quinta": "quinta",
    "br": "bairro", "bairro": "bairro",
    "rot": "rotunda", "rotunda": "rotunda",
    "via": "via", "passeio": "passeio",
    "urb": "urbanizacao", "urbanizacao": "urbanizacao",
    "zona": "zona", "lgd": "lugar", "lugar": "lugar",
    "campo": "campo", "terreiro": "terreiro"
}

# ==========================
# 2. Funções de normalização
# ==========================
def normalizar(texto: str) -> str:
    if not texto:
        return ""
    txt = unidecode.unidecode(str(texto).lower())
    txt = txt.replace(".", "").strip()
    palavras = txt.split()
    palavras = [abreviaturas.get(p, mapa_logradouros.get(p, p)) for p in palavras]
    return " ".join(palavras)

def limpar_nome_rua(nome: str) -> str:
    if not nome:
        return ""
    txt = unidecode.unidecode(str(nome).lower())
    # remover logradouros
    for log in set(mapa_logradouros.values()):
        txt = re.sub(rf"\b{log}\b", "", txt)
    # remover tokens de número
    txt = re.sub(r"\b(n|nr|nº|nro|numero|num)\b", "", txt)
    txt = re.sub(r"\d+", "", txt)
    # expandir abreviaturas
    palavras = txt.split()
    palavras = [abreviaturas.get(p, p) for p in palavras]
    txt = " ".join(palavras)
    txt = re.sub(r"\s+", " ", txt).strip()
    return txt

# ==========================
# 3. Extrair + normalizar partes
# ==========================
def extrair_partes(df: pl.DataFrame) -> pl.DataFrame:
    # Extrair componentes com regex
    df_parts = df.with_columns([
        # logradouro (case-insensitive, aceita abreviações R., Av., Tv.)
        pl.col("MORADA").str.extract(
            r"(?i)^(R\.?|RUA|AV\.?|AVENIDA|TRAV\.?|TRAVESSA|PRAÇA|ESTRADA|BECO|ALAMEDA|LARGO|CALÇADA|ROTUNDA)", 1
        ).alias("logradouro"),

        # nome da rua
        pl.col("MORADA").str.extract(
            r"(?i)^(?:R\.?|RUA|AV\.?|AVENIDA|TRAV\.?|TRAVESSA|PRAÇA|ESTRADA|BECO|ALAMEDA|LARGO|CALÇADA|ROTUNDA)\s+([A-ZÇÃÕÉÓÀ\s]+?)(?:\d|$)", 
            1
        ).str.strip_chars().alias("nome_rua"),

        # número
        pl.col("MORADA").str.extract(r"\s(\d+[A-Z]?)\b", 1).alias("numero"),

        # andar
        pl.col("MORADA").str.extract(r"(\d+º\s*[A-Z]*|R/C)", 1).alias("andar"),

        # código postal
        pl.col("MORADA").str.extract(r"(\d{4}-\d{3})", 1).alias("codigo_postal"),

        # localidade (usar coluna LOCALIDADE quando existir)
        pl.when(pl.col("LOCALIDADE").is_not_null())
          .then(pl.col("LOCALIDADE"))
          .otherwise(pl.col("MORADA").str.extract(r"([A-ZÇÃÕÉÓ]+)$", 1))
          .alias("localidade")
    ])

    # Aplicar normalização
    df_parts = df_parts.with_columns([
        pl.col("logradouro").map_elements(normalizar, return_dtype=pl.Utf8).alias("logradouro_norm"),
        pl.col("nome_rua").map_elements(limpar_nome_rua, return_dtype=pl.Utf8).alias("nome_rua_norm"),
        pl.col("numero").map_elements(normalizar, return_dtype=pl.Utf8).alias("numero_norm"),
        pl.col("andar").map_elements(normalizar, return_dtype=pl.Utf8).alias("andar_norm"),
        pl.col("codigo_postal").map_elements(normalizar, return_dtype=pl.Utf8).alias("codigo_postal_norm"),
        pl.col("localidade").map_elements(normalizar, return_dtype=pl.Utf8).alias("localidade_norm")
    ])
    
    return df_parts



# SAE

In [ ]:
# aplicar extrair_partes às moradas de cada lado
df1_parts = extrair_partes(pl.DataFrame({
    "MORADA": pairs_500["MORADA_df1"],
    "LOCALIDADE": pairs_500["LOCALIDADE_df1"]
}))

df2_parts = extrair_partes(pl.DataFrame({
    "MORADA": pairs_500["MORADA_df2"],
    "LOCALIDADE": pairs_500["LOCALIDADE_df2"]
}))


In [50]:
df1_parts

NameError: name 'df1_parts' is not defined

In [ ]:
def comparar_dataframes(df1: pl.DataFrame, df2: pl.DataFrame, threshold=85):
    results = []
    pdf1, pdf2 = df1.to_pandas(), df2.to_pandas()  # converte para pandas

    for i, row1 in pdf1.iterrows():
        for j, row2 in pdf2.iterrows():
            score_log = fuzz.token_sort_ratio(row1["logradouro_norm"], row2["logradouro_norm"])
            score_rua = fuzz.token_sort_ratio(row1["nome_rua_norm"], row2["nome_rua_norm"])
            score_num = fuzz.token_sort_ratio(str(row1["numero_norm"]), str(row2["numero_norm"]))
            score_loc = fuzz.token_sort_ratio(row1["localidade_norm"], row2["localidade_norm"])

            score_final = (0.3*score_log + 0.4*score_rua + 0.2*score_num + 0.1*score_loc)

            if score_final >= threshold:
                results.append({
                    "id_df1": i, "id_df2": j,
                    "score": round(score_final, 2),
                    "logradouro1": row1["logradouro_norm"], "logradouro2": row2["logradouro_norm"],
                    "nome_rua1": row1["nome_rua_norm"], "nome_rua2": row2["nome_rua_norm"],
                    "numero1": row1["numero_norm"], "numero2": row2["numero_norm"],
                    "localidade1": row1["localidade_norm"], "localidade2": row2["localidade_norm"]
                })
    return pd.DataFrame(results)


In [ ]:
df_matches = comparar_dataframes(df1_parts, df2_parts, threshold=85)

# juntar de volta a distância em metros
df_matches = df_matches.merge(
    pairs_500.reset_index().rename(columns={"index":"id_pair"})[["id_pair","dist_m"]],
    left_on=["id_df1","id_df2"], right_on=["id_pair","id_pair"], how="left"
).drop("id_pair", axis=1)

df_matches


,id_df1,id_df2,score,logradouro1,logradouro2,nome_rua1,nome_rua2,numero1,numero2,localidade1,localidade2,dist_m
0,0,0,92.94,rua,rua,comandante luis pinto da silva,comandante luis pinto,37,37,povoa de lanhoso,povoa de lanhoso,3.333675
1,0,2,92.94,rua,rua,comandante luis pinto da silva,comandante luis pinto,37,37,povoa de lanhoso,povoa de lanhoso,NaN
2,0,3,92.94,rua,rua,comandante luis pinto da silva,comandante luis pinto,37,37,povoa de lanhoso,povoa de lanhoso,NaN
3,0,4,92.94,rua,rua,comandante luis pinto da silva,comandante luis pinto,37,37,povoa de lanhoso,povoa de lanhoso,NaN
4,0,5,92.94,rua,rua,comandante luis pinto da silva,comandante luis pinto,37,37,povoa de lanhoso,povoa de lanhoso,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
662,164,218,85.00,rua,rua,comandante luis pinto silva,comandante luis pinto,70,37,povoa de lanhoso,povoa de lanhoso,NaN
663,164,219,85.00,rua,rua,comandante luis pinto silva,comandante luis pinto,70,37,povoa de lanhoso,povoa de lanhoso,NaN
664,164,220,85.00,rua,rua,comandante luis pinto silva,comandante luis pinto,70,37,povoa de lanhoso,povoa de lanhoso,NaN
665,164,221,85.00,rua,rua,comandante luis pinto silva,comandante luis pinto,70,37,povoa de lanhoso,povoa de lanhoso,NaN


In [ ]:
df_matches[df_matches["dist_m"].notna()]


,id_df1,id_df2,score,logradouro1,logradouro2,nome_rua1,nome_rua2,numero1,numero2,localidade1,localidade2,dist_m
0,0,0,92.94,rua,rua,comandante luis pinto da silva,comandante luis pinto,37,37,povoa de lanhoso,povoa de lanhoso,3.333675
222,1,1,100.00,avenida,avenida,da capela,da capela,147,147,povoa de lanhoso,povoa de lanhoso,3.479725
299,77,77,88.00,rua,rua,comandante luis pinto,comandante luis pinto,123,37,povoa de lanhoso,povoa de lanhoso,58.619899
608,164,164,85.00,rua,rua,comandante luis pinto silva,comandante luis pinto,70,37,povoa de lanhoso,povoa de lanhoso,37.977748


In [ ]:
df_matches_valid = df_matches[df_matches["dist_m"].notna()].reset_index(drop=True)
df_matches_valid


,id_df1,id_df2,score,logradouro1,logradouro2,nome_rua1,nome_rua2,numero1,numero2,localidade1,localidade2,dist_m
0,0,0,92.94,rua,rua,comandante luis pinto da silva,comandante luis pinto,37,37,povoa de lanhoso,povoa de lanhoso,3.333675
1,1,1,100.00,avenida,avenida,da capela,da capela,147,147,povoa de lanhoso,povoa de lanhoso,3.479725
2,77,77,88.00,rua,rua,comandante luis pinto,comandante luis pinto,123,37,povoa de lanhoso,povoa de lanhoso,58.619899
3,164,164,85.00,rua,rua,comandante luis pinto silva,comandante luis pinto,70,37,povoa de lanhoso,povoa de lanhoso,37.977748


In [ ]:
m1 = psutil.Process(os.getpid()).memory_info().rss / 1024**2
print("Tempo Polars:", round(time.time() - start, 2), "segundos")
print("Memória Polars:", round(m1 - m0, 2), "MB")

Tempo Polars: 93.48 segundos
Memória Polars: 383.24 MB
